In [1]:
import requests # from urllib.request import urlopen
from bs4 import BeautifulSoup # HTTP Response -> HTML 

import pandas as pd

from datetime import datetime

import time 
import re #정규 표현식 활용

In [2]:
#망고플레이트에서 음식을 키워드로 맛집 리스트 가져오는 function
def get_rest_list(keyword):
    
    #페이지, 맛집리스트 초기화
    pagenum = 1
    restaurant_list = []
    
    #page별로 while 루프 돌려서 상호명 뽑아오기
    while pagenum <= 2: 
        try:
            url = 'https://www.mangoplate.com/search/?keyword='+keyword+'&page='+str(pagenum)#keyword를 input으로 망고플레이트 url 생성

            session = requests.Session()
            headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
            }
            html = session.get(url, headers=headers).content 
            source = BeautifulSoup(html, 'html.parser')

            restaurants = source.find_all('h2', {'class' : 'title'}) #h2태그의 class 'title' => 상호명

            for restaurant in restaurants:#텍스트 깔끔하게 처리 및 리스트에 상호명 추가 루프

                n_restaurant = re.sub('\s', '', restaurant.get_text()) #텍스트 중간에 빈칸 없애기
                restaurant_list.append(n_restaurant)#리스트에 상호명 추가
        except:
            print(pagenum,'페이지 까지의 결과입니다.')
        
        pagenum += 1
        
    return restaurant_list #keyword 관련 맛집 리스트 생성

In [3]:
cnoodlelist = get_rest_list('냉면') #냉면 맛집 리스트 생성

In [4]:
cnoodlelist

['허브족발(본점)',
 '윤밀원',
 '김진목삼',
 '진미평양냉면',
 '서관면옥',
 '광릉한옥집',
 '광화문국밥',
 '춘향미엔',
 '우래옥(을지로점)',
 '을지면옥',
 '설눈',
 '봉밀가',
 '봉피양(방이점)',
 '미엔아이',
 '윤선희의평양냉면양각도',
 '청기와타운',
 '서평면옥',
 '몽탄',
 '정인면옥',
 '강서면옥',
 '오장동흥남집(본점)',
 '거대갈비',
 '평안도상원냉면',
 '명지원이동갈비',
 '벽제갈비(신촌점)',
 '공단떡볶이',
 '고명집',
 '우밀면옥',
 '단천식당',
 '명인등심(압구정점)',
 '평남면옥',
 '서정가야밀면',
 '더진미평냉',
 '명인등심(삼성직영점)',
 '영동냉면',
 '을밀대(본점)',
 '능라도(강남점)',
 '능라도(분당점)',
 '외래향(서울대점)',
 '옥돌현옥']

In [5]:
def get_mango_info(restaurant):#상호명을 input으로 관련 정보 받아오는 function

    mango_info = {} #빈 dict 생성

    url = 'https://www.mangoplate.com/search/?keyword='+restaurant #상호명으로 망고플레이트 검색 url

    session = requests.Session()
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(url, headers=headers).content 
    source = BeautifulSoup(html, 'html.parser')

    rest_code = source.find('a',{'class':'only-desktop_not'}).attrs['href']#검색결과에서 첫번째 결과의 고유 코드 따오기 
    newurl = 'https://www.mangoplate.com'+rest_code #고유 코드 활용한 url 주소 생성
    newhtml = session.get(newurl, headers=headers).content 
    newsource = BeautifulSoup(newhtml, 'html.parser')
    
    rate = float(re.sub('\s','',newsource.find('strong',{'class':'rate-point'}).get_text()))#평점이 속한 태그에서 텍스트 가져와 float로 바꾸기 (나중에 계산 용이)

    rest_table = newsource.find('table',{'class':'info'})#관련 정보가 있는 테이블 경로

    address = rest_table.find_all('td')[0].find_all('span')[1].get_text()#주소
    phone = rest_table.find_all('td')[1].get_text()#전화번호
    kind = re.sub('\s','',rest_table.find_all('td')[2].get_text())#음식 종류(정규표현식 활용하여 깔끔하게 정리)
    price = rest_table.find_all('td')[3].get_text()#가격대

    #각각의 정보를 만든 dict의 key값의 value로 추가
    mango_info['name'] = restaurant
    mango_info['rate'] = rate
    mango_info['address'] = address
    mango_info['phone'] = phone
    mango_info['kind'] = kind
    mango_info['price'] = price
    
    return mango_info #하나의 dict를 최종 값으로 return



In [8]:
def get_naver_info(restaurant):
    
    naver_info = {}
        
    url = "https://search.naver.com/search.naver?where=nexearch&sm=tab_jum&query=" + restaurant
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')
    
    try:
        reviewcount = int(source.find_all('span', {'class': 'count'})[1].get_text())
        rate = float(source.find_all('div', {'class': 'star_area'})[0].find('em').get_text())
    except:
        reviewcount = 'N/A'
        rate = 'N/A'
    
    naver_info['reviewcount'] = reviewcount
    naver_info['rate'] = rate
    
    return naver_info


    

In [9]:
#DataFrame을 위한 빈 리스트 생성

mango_name = []
mango_rate = []
mango_address = []
mango_phone = []
mango_kind = []
mango_price = []
naver_reviewcount = []
naver_rate = []

#음식정 상호명 리스트를 활용하여 각각의 점포의 정보를 받아오는 for loop
for restaurant in cnoodlelist:
    
    mango_info = get_mango_info(restaurant)
    naver_info = get_naver_info(restaurant)
    #위에서 정의한 망고플레이트 음식점 정보를 하나의 dict로 가져옴
    
    #각각의 리스트에 고유 정보 추가
    mango_name.append(mango_info['name'])
    mango_rate.append(mango_info['rate'])
    mango_address.append(mango_info['address'])
    mango_phone.append(mango_info['phone'])
    mango_kind.append(mango_info['kind'])
    mango_price.append(mango_info['price'])
    naver_reviewcount.append(naver_info['reviewcount'])
    naver_rate.append(naver_info['rate'])

    

In [10]:
#만든 리스트로 DataFrame 생성

restaurant_df = pd.DataFrame({'음식점':mango_name,
                         '망고평점': mango_rate,
                         '네이버평점': naver_rate,
                         '네이버리뷰수': naver_reviewcount,     
                         '주소': mango_address,
                         '전화': mango_phone,
                         '종류': mango_kind,
                         '가격': mango_price})

restaurant_df

,음식점,망고평점,네이버평점,네이버리뷰수,주소,전화,종류,가격
0,허브족발(본점),4.6,N/A,N/A,서울시 영등포구 당산동6가 237-38,02-2633-4339,고기요리,3만원-4만원
1,윤밀원,4.6,N/A,N/A,경기도 성남시 분당구 정자동 223-9,031-714-8388,국수/면요리,만원 미만
2,김진목삼,4.6,4.69,246,서울시 종로구 필운동 154,02-929-2929,고기요리,만원-2만원
3,진미평양냉면,4.5,N/A,N/A,서울시 강남구 논현동 115-10,02-515-3469,국수/면요리,만원-2만원
4,서관면옥,4.5,N/A,N/A,서울시 서초구 서초동 1675-5,02-521-9945,국수/면요리,만원-2만원
5,광릉한옥집,4.5,4.43,613,경기도 남양주시 진접읍 부평리 602-1,031-574-6630,고기요리,만원-2만원
6,광화문국밥,4.5,N/A,N/A,서울시 중구 정동 1-48,02-738-5688,탕/찌개/전골,만원-2만원
7,춘향미엔,4.5,N/A,N/A,서울시 성동구 행당동 299-1,02-2281-9031,딤섬/만두,만원 미만
8,우래옥(을지로점),4.4,N/A,N/A,서울시 중구 주교동 118-1,02-2265-0151,국수/면요리,만원-2만원
9,을지면옥,4.4,4.35,944,서울시 중구 입정동 161,02-2266-7052,국수/면요리,만원-2만원
